In [ ]:
using cfgrib
using cfgrib: DataSet, Variable

using DataStructures

Another randomly placed note: in Julia you would typically directly export the types and functions you want to make available out of you package, however I haven't done this yet so you either have to call `cfgrib.whatever` or do `using cfgrid: whatever`.

In [ ]:
dir_tests = abspath(joinpath(dirname(pathof(cfgrib)), "..", "test"))
dir_testfiles = abspath(joinpath(dir_tests, "sample-data"))
test_file = joinpath(dir_testfiles, "era5-levels-members.grib");

In [ ]:
dataset = DataSet(test_file);

## AxisArrays

In [ ]:
using Pkg

if !("AxisArrays" in keys(Pkg.installed()))
    Pkg.add("AxisArrays")
end

In [ ]:
using AxisArrays

Important note: AxisArrays does not (natively) have the same notion of xarray Datasets, which makes this a bit awkward, so we create a wrapper type:

In [ ]:
mutable struct AxisArrayWrapper
    dimensions::OrderedDict
    datasets::T where T <: NamedTuple
    attributes::OrderedDict
    encoding::Dict
    
    AxisArrayWrapper() = new() #  Allow undefined initialisation
end

Pretty simple, we just store the axisarrays in a named tuple. To have similar notation to xarray, we then make indexing go into the named tuple:

In [ ]:
Base.getindex(obj::AxisArrayWrapper, key) = obj.datasets[key]
Base.keys(obj::AxisArrayWrapper) = keys(obj.dataset) #  Kinda stupid as you can't access the axis

Now to figure out how to actually build this object:

In [ ]:
dataset.dimensions

In [ ]:
dataset.variables

Not sure if there's a better way to do this, here I just pick out all the multidimensional variables:

In [ ]:
[size(v.data) for v in values(dataset.variables)]

In [ ]:
multidimensional_idx =  (
    [size(v.data) for v in values(dataset.variables)]
    .|> length
    .|> x -> x > 1)

Another random sidenote: in python you can easily chain method calls together like `a.b().c().d()...`, in Julia you would need to do something like `d(c(b(a)))`, which is a pain to read, counting nested brackets is not fun.

So, julia has a pipe operator `|>` which can be used to chain functions together, it also has an element-wise version of this with a dot in front `.|>` (nested random sidenote: dots automatically broadcast over each element in situations where it is unambiguous, e.g. `println.([1,2,3])` calls printline on each element).

So above we get the size of each of the variables, pipe this into length element wise, then pipe that array of lengths element wise into an anonymous function, which returns a bitarray of variables with more than one dimension.

And assume that these are the only dimensions that we need:

In [ ]:
multidimensional_keys = collect(keys(dataset.variables))[multidimensional_idx]

Here you can pull out the dimensions we'll use to create our axis:

In [ ]:
shared_dimensions = [dataset.variables[k] for k in keys(dataset.dimensions)]

And here we define the actual axis types used by AxisArrays:

In [ ]:
shared_axis = [Axis{Symbol(k)}(dataset.variables[k].data) for k in keys(dataset.dimensions)]

Now we can test creating an AxisArray for one of the multidimensional variables:

In [ ]:
AxisArray(cfgrib.convert(Array, dataset.variables["t"].data), shared_axis...)

And finally bringing it all together:

In [ ]:
function convert(::Type{AxisArrayWrapper}, dataset::DataSet)
    res = AxisArrayWrapper()
    res.dimensions = dataset.dimensions
    res.attributes = dataset.attributes
    res.encoding = dataset.encoding
    
    multidimensional_idx =  (
        [size(v.data) for v in values(dataset.variables)]
        .|> length
        .|> x -> x > 1)
    multidimensional_keys = collect(keys(dataset.variables))[multidimensional_idx]
    multidimensional_values = [dataset.variables[k] for k in multidimensional_keys]
    
    shared_dimensions = [dataset.variables[k] for k in keys(dataset.dimensions)]
    shared_axis = [Axis{Symbol(k)}(dataset.variables[k].data) for k in keys(dataset.dimensions)]
    
    res.datasets = NamedTuple{Tuple(Symbol.(multidimensional_keys))}((
        AxisArray(cfgrib.convert(Array, dataset.variables[k].data), shared_axis...)
        for k in multidimensional_keys
    ))

    return res
end

In [ ]:
ds = convert(AxisArrayWrapper, dataset);

If you make the mistake of removing `;` you'll see a truly horrific print output, which I haven't had time to improve. But the base functionality is here:

In [ ]:
ds[:t] #  See the AcisArray
#  This shold really have a limit in the IOContext so it prints something sensible
#  easy fix though - https://stackoverflow.com/questions/40788316/julia-limited-printing-of-large-arrays

In [ ]:
ds[:t][number = 1] #  Confusingly this is the INDEX of number, so a number value of 0

In [ ]:
ds[:t][0 .. 0] #  This is the slightly odd syntax to access 

There is a lot more you can do with AxisArrays, as shown here: https://github.com/JuliaArrays/AxisArrays.jl

## DimensionalData

We can take a similar approach to create an interface for DimensionalData.jl

In [ ]:
if !("DimensionalData" in keys(Pkg.installed()))
    Pkg.add("DimensionalData")
end

using DimensionalData

In [ ]:
multidimensional_idx =  (
    [size(v.data) for v in values(dataset.variables)]
    .|> length
    .|> x -> x > 1)

In [ ]:
multidimensional_keys = collect(keys(dataset.variables))[multidimensional_idx]

In [ ]:
shared_dimensions = [dataset.variables[k] for k in keys(dataset.dimensions)]

In [ ]:
shared_axis = [Dim{Symbol(k)}(dataset.variables[k].data) for k in keys(dataset.dimensions)]

In [ ]:
DimensionalArray(cfgrib.convert(Array, dataset.variables["t"].data), Tuple(shared_axis))

In [ ]:
mutable struct DimensionalArrayWrapper
    dimensions::OrderedDict
    datasets::T where T <: NamedTuple
    attributes::OrderedDict
    encoding::Dict
    
    DimensionalArray() = new() #  Allow undefined initialisation
end

In [ ]:
function convert(::Type{DimensionalArrayWrapper}, dataset::DataSet)
    res = AxisArrayWrapper()
    res.dimensions = dataset.dimensions
    res.attributes = dataset.attributes
    res.encoding = dataset.encoding
    
    multidimensional_idx =  (
        [size(v.data) for v in values(dataset.variables)]
        .|> length
        .|> x -> x > 1)
    multidimensional_keys = collect(keys(dataset.variables))[multidimensional_idx]
    multidimensional_values = [dataset.variables[k] for k in multidimensional_keys]
    
    shared_dimensions = [dataset.variables[k] for k in keys(dataset.dimensions)]
    shared_axis = [Dim{Symbol(k)}(dataset.variables[k].data) for k in keys(dataset.dimensions)]
    
    res.datasets = NamedTuple{Tuple(Symbol.(multidimensional_keys))}((
        DimensionalArray(cfgrib.convert(Array, dataset.variables[k].data), Tuple(shared_axis))
        for k in multidimensional_keys
    ))

    return res
end

In [ ]:
convert(DimensionalArrayWrapper, dataset)